In [148]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

from scipy import stats
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
from pylab import rcParams

In [168]:
respeck = pd.read_feather("respeck.feather")
respeck

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,activity_type,activity_code,subject_id
0,1.664890e+12,0.125977,-1.205872,0.143982,16.125000,6.187500,9.515625,Climbing stairs,12,s1732873
1,1.664890e+12,0.104004,-0.896057,0.059509,15.468750,6.812500,18.062500,Climbing stairs,12,s1732873
2,1.664890e+12,0.087402,-1.044495,0.060974,1.953125,-5.453125,15.625000,Climbing stairs,12,s1732873
3,1.664890e+12,0.058594,-0.935364,0.096619,-4.015625,-5.156250,10.718750,Climbing stairs,12,s1732873
4,1.664890e+12,-0.037842,-0.858215,0.167419,-9.843750,-7.828125,9.671875,Climbing stairs,12,s1732873
...,...,...,...,...,...,...,...,...,...,...
832639,1.664892e+12,0.297119,-0.773499,0.205994,54.359375,-23.093750,8.859375,Walking at normal speed,1,s2342658
832640,1.664892e+12,0.097412,-0.588928,0.104431,39.812500,-10.609375,3.062500,Walking at normal speed,1,s2342658
832641,1.664892e+12,-0.062744,-0.577698,-0.086975,6.156250,-14.250000,3.203125,Walking at normal speed,1,s2342658
832642,1.664892e+12,-0.207764,-0.866516,0.027771,-10.062500,-15.343750,22.171875,Walking at normal speed,1,s2342658


In [169]:

respeck.drop(columns=['timestamp','activity_type','subject_id'], axis=1, inplace=True)

In [170]:

split = len(respeck)/3*2
train_respeck = respeck.loc[0:split-1,:]
test_respeck = respeck.loc[split:,:]
print(train_respeck.loc[:,'activity_code'].drop_duplicates())
print(test_respeck.loc[:,'activity_code'].drop_duplicates())

0        12
751      13
1502     31
2252      7
3003      2
3763      8
4513      6
5263      9
6013     11
6763      5
7513      4
8263      0
9013    100
9763      1
Name: activity_code, dtype: int64
555096     12
555424     13
556196     31
556959      7
557734      2
558503      8
559271      6
560046      9
560813     11
561572      5
562345      4
563114      0
563888    100
564651      1
Name: activity_code, dtype: int64


In [171]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
def create_dataset(r, time_steps, step,train=True):
    global enc
    x_list, y_list = [], []
    print(r.tail())
    for i in range(0, len(r) - time_steps, step):
        v = r.iloc[i:(i + time_steps)]
        labels = v.pop('activity_code')
        
        x_list.append(v.values)
        y_list.append(stats.mode(labels)[0][0])

    x_train = np.array(x_list)
    y_train = np.array(y_list).reshape(-1, 1)
    x_train.shape

    if(train):
        enc = enc.fit(y_train)    # Call 'fit' with appropriate arguments before using this estimator.
    y_train = enc.transform(y_train)
    return (x_train,y_train)

def create_model(trainX, trainy):
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # evaluate model
    return model

In [172]:
time_steps = 50
step = 10

x_train,y_train = create_dataset(train_respeck,time_steps, step)

model = create_model(trainX=x_train,trainy=y_train)



         accel_x   accel_y   accel_z     gyro_x     gyro_y     gyro_z  \
555091 -0.507324 -0.778625  0.078796 -29.468750 -24.937500   6.484375   
555092 -0.380615 -0.660950  0.240417   1.890625 -16.296875 -13.625000   
555093 -0.306641 -0.531311  0.095398   2.984375  -3.687500  -5.625000   
555094 -0.287109 -0.437317  0.124695 -14.265625 -10.812500   4.406250   
555095 -0.290039 -0.458557  0.236511 -10.781250  -8.421875   7.437500   

        activity_code  
555091             12  
555092             12  
555093             12  
555094             12  
555095             12  


In [ ]:
x_train

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [174]:
model.fit(x_train, y_train, epochs=40, batch_size=32, verbose=1)

Epoch 1/40
1735/1735 [==============================] - 15s 8ms/step - loss: nan - accuracy: 0.1017
Epoch 2/40
1735/1735 [==============================] - 14s 8ms/step - loss: nan - accuracy: 0.0729
Epoch 3/40
1735/1735 [==============================] - 15s 9ms/step - loss: nan - accuracy: 0.0729
Epoch 4/40
1735/1735 [==============================] - 15s 8ms/step - loss: nan - accuracy: 0.0729
Epoch 5/40
1735/1735 [==============================] - 15s 9ms/step - loss: nan - accuracy: 0.0729
Epoch 6/40
1735/1735 [==============================] - 15s 9ms/step - loss: nan - accuracy: 0.0729
Epoch 7/40
1735/1735 [==============================] - 15s 8ms/step - loss: nan - accuracy: 0.0729
Epoch 8/40
1735/1735 [==============================] - 14s 8ms/step - loss: nan - accuracy: 0.0729
Epoch 9/40
1735/1735 [==============================] - 15s 8ms/step - loss: nan - accuracy: 0.0729
Epoch 10/40
1051/1735 [=================>............] - ETA: 5s - loss: nan - accuracy: 0.0728

KeyboardInterrupt: 